In [1]:
from helpers.dataset_preprocessing import create_dataset, extract_patches, compute_input_features, compute_output_features
from helpers.dataset_postprocessing import unpatch
from helpers.feature_extractors import extract_features, extract_features_2d, value_to_class, value_to_2d_class
from helpers.visualization_helpers import *

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as n

%matplotlib inline

In [2]:
# GLOBAL VARIABLES
ROOT_DIR = "training/"
TOTAL_IMAGES = 10 # Number of images to load
TRAIN_FRACTION = 0.8 # Percentage of images used for training
ANGLE_STEP = 15 # Gotta be 360/ANGLE_STEP needs to be an integer
FLIP = True # Flag to signal if flipped  versions of rotated images should also be created
PATCH_SIZE = 8
PATCH_TRANSLATION = 0 # WARNING: this quickly explodes to enormous amount of data if small patch_translation is selected.
FOREGROUND_THRESHOLD = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
ORIGINAL_IMAGE_WIDTH = 400
ORIGINAL_IMAGE_HEIGHT = 400
NUM_CHANNELS = 6
THEANO = False

In [ ]:
dataset = create_dataset(ROOT_DIR, TOTAL_IMAGES, TRAIN_FRACTION, rotation_angle=ANGLE_STEP, flip=FLIP)

Original loaded dataset size: 10
Creating train dataset...
Number of rotations: 24
Flipping: True
Creating test dataset...
Number of rotations: 24
Flipping: True
Created train dataset size: 3760
Created test dataset size: 20


In [ ]:
patches = extract_patches(dataset, PATCH_SIZE, PATCH_TRANSLATION)

In [ ]:
def dont_extract(input):
    return input

In [ ]:
input_features = compute_input_features(patches[0:2], extract_features) # train_img_patches and test_img_patches
output_features = compute_output_features(patches[2:4], value_to_2d_class, FOREGROUND_THRESHOLD) # train_gt_img_patches and test_gt_img_patches
 
from skimage import img_as_float
    
X_train = img_as_float(input_features[0])
Y_train = output_features[0].astype(np.float32)
X_test = img_as_float(input_features[1])
Y_test = output_features[1].astype(np.float32)
 

In [ ]:
from __future__ import print_function
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import load_model
from keras.optimizers import SGD
from keras import backend as K

batch_size = 128
nb_classes = 2


# ********** Tuning parameters: (See Network architecture as well)

# Epochs to be trained
nb_epoch = 10
# number of convolutional filters to use
nb_filters = 64
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (4, 3)

"""
if "image_dim_ordering": is "th" and "backend": "theano", your input_shape must be (channels, height, width)
if "image_dim_ordering": is "tf" and "backend": "tensorflow", your input_shape must be (height, width, channels)
"""
if THEANO:
    input_shape = (NUM_CHANNELS, PATCH_SIZE, PATCH_SIZE)
else:
    input_shape = (PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS)

def train_cnn(model_name='test.h5'):


    # **************** DEFINE THE MODEL ARCHITECTURE *******************

    model = Sequential()

    # Convolution layer with rectified linear activation
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],border_mode='same',input_shape=input_shape))
    model.add(Activation('relu'))

    # Second convolution
    model.add(Convolution2D(nb_filters, kernel_size[1], kernel_size[0]))
    model.add(Activation('relu'))

    model.add(Dropout(0.25))

    # Third convolution
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[0]))
    model.add(Activation('relu'))

    # Pooling and dropout
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    # Full-connected layer
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))

    model.add(Dense(1024))
    model.add(Activation('relu'))

    # Dropout to avoid overfitting
    model.add(Dropout(0.25))

    model.add(Dense(1024))
    model.add(Activation('relu'))

    # Dropout to avoid overfitting
    model.add(Dropout(0.5))

    #Fully-connected layer to ouptut the resulting class
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['fmeasure'])


    #class_weight = auto??
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, class_weight='auto', verbose=1, validation_data=(X_test, Y_test))


    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

    model.save('models/' + model_name)

In [ ]:
train_cnn()